In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import numpy as np

def scheduler_exp(epoch,lr):
    if epoch < 1 :
        return lr
    else:
        return lr*np.exp(-1.0)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

from tensorflow.keras.callbacks import LearningRateScheduler

def scheduler_exp(epoch,lr):
    if epoch < 1:
        return lr
    else :
        return lr*np.exp(-1.0)
    
model = Sequential([Dense(10)])
model.compile(optimizer=SGD(), loss='mse')

print('최초 learining rated ',round(model.optimizer.lr.numpy(),5))

In [ ]:
lr_scheduler = LearningRateScheduler(scheduler_exp, verbose=1)

history = model.fit(np.arange(100).reshape(5,20), np.zeros(5),epochs=15, callbacks=[lr_scheduler],verbose=1)

In [ ]:
np.arange(100).reshape(5,20)

In [ ]:
np.zeros(5)

In [ ]:
epoch_list = range(30)

In [ ]:
def scheduler_exp(epoch):
    initial_lr = 0.01
    if epoch < 1:
        return initial_lr
    else:
        return initial_lr*np.exp(-1.0)**epoch
    
model = Sequential([Dense(10)])
model.compile(optimizer=SGD(),loss='mse')

lr_scheduler = LearningRateScheduler(scheduler_exp, verbose=1)
history = model.fit(np.arange(100).reshape(5,20), np.zeros(5),epochs=15, callbacks=[lr_scheduler],verbose=1)

In [ ]:
lr_list = [scheduler_exp(epoch) for epoch in range(30)]
lr_list
epoch_list = range(30)
epoch_list

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_scheduler(epochs_list, lr_list, title=None):
    plt.figure(figsize=(6,4))
    plt.plot(epochs_list, lr_list)
    plt.xlabel('epochs')
    plt.ylabel('lr')
    plt.title(title)

In [ ]:
plot_scheduler(epoch_list, lr_list, 'exponentially decay')

import tensorflow as tf

def step_decay(epoch):
    inital_lr = 0.1
    drop = 0.5
    epochs_drop = 5.0
    lr = inital_lr * (drop**np.floor(epoch/epochs_drop))
    return lr

lr_list = [step_decay(epoch) for epoch in epoch_list]
plot_scheduler(epoch_list, lr_list, title='Step Decay')

In [ ]:
model = Sequential([Dense(10)])
model.compile(optimizer=SGD(),loss='mse')

lr_scheduler = LearningRateScheduler(step_decay, verbose=1)
history = model.fit(np.arange(100).reshape(5,20), np.zeros(5),epochs=15,
                   callbacks=[lr_scheduler],verbose=1)

In [ ]:
cos_decay = tf.keras.experimental.CosineDecay(initial_learning_rate=1e-2, decay_steps=30, alpha=1e-2)

In [ ]:
type(cos_decay)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_scheduler(epochs_list, lr_list, title=None):
    plt.figure(figsize=(6,4))
    plt.plot(epochs_list, lr_list)
    plt.xlabel('steps')
    plt.ylabel('lr')
    plt.title(title)
    
cos_decay = tf.keras.experimental.CosineDecay(initial_learning_rate=1e-2,decay_steps=30,alpha=1e-2)
steps_list = range(0,30)

lr_list = cos_decay(steps_list)
plot_scheduler(steps_list, lr_list, title='Cosine Decay')

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Dense(10)])

model.compile(tf.keras.optimizers.Adam(learning_rate = cos_decay), loss='mse')
history = model.fit(np.arange(100).reshape(5,20),np.zeros(5),epochs=15,verbose=1)

In [ ]:
decay_steps=30
alpha = 0.0
initial_learning_rate=0.01

def decayed_learning_rate(step):
    step = min(step, decay_steps)
    cosine_decay = 0.5 * (1+np.cos(np.pi*step/decay_steps))
    decayed = (1-alpha)*cosine_decay+alpha
    return initial_learning_rate * decayed

steps_list = range(0,30)
lr_list = [decayed_learning_rate(step) for step in steps_list]
plot_scheduler(steps_list, lr_list, 'Cosine Decay')

In [ ]:
from tensorflow.keras.experimental import CosineDecayRestarts

cos_decay_restarts = CosineDecayRestarts(initial_learning_rate=0.01, first_decay_steps= 10, t_mul=1, m_mul=0.9, alpha=0)
steps_list = range(0,120)

lr_list = cos_decay_restarts(steps_list)

plot_scheduler(steps_list, lr_list, 'Cosine Decay Restarts')

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Dense(10)])

model.compile(tf.keras.optimizers.Adam(learning_rate=cos_decay_restarts),loss='mse')
history = model.fit(np.arange(100).reshape(5,20),np.zeros(5),epochs=15, verbose=1)

In [ ]:
def lrfn(epoch):
    LR_START = 1e-5
    LR_MAX = 1e-2
    LR_RAMPUP_EPOCHS = 3
    LR_SUSTAIN_EPOCHS=3
    LR_STEP_DECAY=0.75
    
    def calc_fn(epoch):
        if epoch < LR_RAMPUP_EPOCHS:
            lr = ((LR_MAX-LR_START)/LR_RAMPUP_EPOCHS)*epoch + LR_START
        elif epoch < LR_RAMPUP_EPOCHS+LR_SUSTAIN_EPOCHS:
            lr = LR_MAX
        else:
            lr = LR_MAX * LR_STEP_DECAY **((epoch-LR_RAMPUP_EPOCHS-LR_SUSTAIN_EPOCHS)//2)
        print('epoch:',epoch, 'lr:',lr)
        return lr
    return calc_fn(epoch)

In [ ]:
import tensorflow as tf
import numpy as np

model = tf.keras.models.Sequential([tf.keras.layers.Dense(10)])
model.compile(tf.keras.optimizers.SGD(),loss='mse')
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lrfn)

history = model.fit(np.arange(100).reshape(5,20), np.zeros(5),epochs=30, callbacks=[lr_scheduler],verbose=1)

In [ ]:
LR_START = 1e-5
LR_MAX = 1e-2
LR_RAMPUP_EPOCHS = 3
LR_SUSTAIN_EPOCHS=3
LR_STEP_DECAY=0.75

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = ((LR_MAX-LR_START)/LR_RAMPUP_EPOCHS)*epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = LR_MAX * LR_STEP_DECAY **((epoch-LR_RAMPUP_EPOCHS-LR_SUSTAIN_EPOCHS)//10)
    return lr
epochs_list = range(0,120)
lr_list = [lrfn(epoch) for epoch in epochs_list]

plot_scheduler(epochs_list, lr_list, title="WarmUp")